In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.metrics import roc_auc_score, auc
from sklearn.model_selection import train_test_split
import gc #garbage collector
import sys
import os
from pandas.api.types import CategoricalDtype
import xgboost as xgb
from itertools import product
from xgboost import XGBRegressor
from statistics import stdev 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from datetime import date
import pickle
import pyodbc
import time
import tqdm
import copy
from sklearn2pmml import PMMLPipeline
from sklearn2pmml import make_pmml_pipeline
from sklearn2pmml import Pipeline
from sklearn.pipeline import Pipeline
from sklearn2pmml import sklearn2pmml


In [221]:
sns.set()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [230]:
df = pd.read_csv('model_data_test_2.csv')

In [258]:
def create_na_report(df):
    """
    create a report of variables whose values are NA across all table, and eliminate them
    """
    print("\nDataset has {} entries and {} features".format(*df.shape))
    na_report = df.isna().sum()                          #create a list of columns and sum of null values
    na_report = pd.DataFrame(na_report, index=None)     #convert list to a dataframe
    na_report.reset_index(inplace=True)                  #reset index to range(0 : len(df)-1)
    na_report.columns = ["variable", "na_count"]        #set column names
    na_report["perc"] = na_report["na_count"]/df.shape[0]           #add a new column which is percentage of null vales by column name
    gc.collect()                                                #garbage collecter for memory efficiency
    return na_report

In [259]:
def create_dtype_report(df):
    """
    define the following reports:
    num_cols: indicating which variables are numeric
    char_cols: indicating which variables are of type character
    """
    df.replace({True: 1, False: 0}, inplace=True)
    df_cols = df.columns
    num_cols = list(df._get_numeric_data().columns)
    char_cols = list(set(df_cols) - set(num_cols))
    gc.collect()
    return num_cols, char_cols

In [260]:
def eliminate_missing(df, na_report, max_threshold):
    ''' 
    # define maximum threshold of missing percentage
    # eliminate columns that do not fit the threshold
    '''
    cols_before = df.columns.tolist()
    cols_ = df.columns[df.columns.isin(list(na_report[na_report['perc'] < max_threshold]['variable']) or df.columns.isin(list(key_var)))]
    df = df[cols_]
    print(len(cols_before)-len(cols_),"columns eliminated due to missing values")
    del cols_, cols_before
    gc.collect()
    return df

In [261]:
def eliminate_single_unique(df):
    ''' 
    # eliminate columns having a single unique value 
    '''
    cols_before = df.columns.tolist()
    unique_report = df.nunique()                          
    unique_report = pd.DataFrame(unique_report, index=None)    
    unique_report.reset_index(inplace=True)
    unique_report.columns = ["variable", "number_unique"]        #set column names
    unique_report = unique_report[~unique_report['variable'].isin(key_var)]  #discard key variables  
    u_cols = df.columns[~df.columns.isin(list(unique_report[unique_report['number_unique'] == 1]['variable']))]
    df = df[u_cols]
    print(len(cols_before)-len(u_cols),"columns eliminated due to single unique values, namely:",list(set(cols_before) - set(u_cols)))
    del unique_report, u_cols, cols_before
    gc.collect()
    return df


In [262]:
def cal_auc_and_gini(df, sample='train', pred_name='predicted'):
    temp = df.copy()
    auc_ = roc_auc_score(temp.loc[temp['sample'] == sample]['actual'], temp.loc[temp['sample'] == sample][pred_name])
    gini_ = round((2*auc_ - 1)*100, 2)
    auc_ = round(auc_*100, 2)
    del temp
    gc.collect()
    return auc_, gini_

In [263]:
def expand_grid(dictionary):
   """Create a dataframe from every combination of given values."""
   return pd.DataFrame([row for row in product(*dictionary.values())],
                       columns=dictionary.keys())

In [264]:
def set_split_index(df, splitting_perc, seed, min_date):
    # allocate Development, and OOT validation data
    oot_index = df.loc[df['APPLICATION_DATE'] >= min_date].index.values  ##REPORT_DATE 
    dev_index = df.loc[df['APPLICATION_DATE'] < min_date].index.values  
    
    y_all = df['TARGET'].iloc[dev_index]
    
    train_index, test_index, y_train, y_test = train_test_split(dev_index, y_all, test_size=splitting_perc, stratify=y_all, random_state=seed)   
    print("\nShape of dataframes; train: {}, test: {}, oot: {}".format(train_index.shape,test_index.shape,oot_index.shape))
    print("\nPercentage of dataframes; train: {:.2%}, test: {:.2%}, oot: {:.2%}".format(len(train_index)/len(df),len(test_index)/len(df),len(oot_index)/len(df)))
    return train_index, test_index, oot_index

In [265]:
def split_df_by_index(df, key_var, train_index, test_index, oot_index):
    # first get TARGET arrays
    y_train, y_test, y_oot = df['TARGET'].iloc[train_index], df['TARGET'].iloc[test_index], df['TARGET'].iloc[oot_index]
    
    # get key values apart including TARGET
    cols_ = df.columns[df.columns.isin(list(key_var))] #including TARGET
    df_key_var = df[cols_]  #save key values
    df_n = df.drop(cols_, axis=1)
    
    # set dataframes
    X_train, X_test, X_oot = df_n.iloc[train_index], df_n.iloc[test_index], df_n.iloc[oot_index]
    train_keys, test_keys, oot_keys = df_key_var.iloc[train_index], df_key_var.iloc[test_index], df_key_var.iloc[oot_index]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    doot = xgb.DMatrix(X_oot, label=y_oot)
    del df_n
    gc.collect()
    return dtrain, dtest, doot, y_train, y_test, y_oot, train_keys, test_keys, oot_keys

In [266]:
def create_calibration_df(predictions, sample):
    df_temp_sum = predictions.loc[predictions['sample'] == sample].copy()
    df_temp_sum.loc[:,'count'] = 1
    df_temp_sum['row_score'] = np.log(df_temp_sum["predicted"]/(1-df_temp_sum["predicted"]))
    gc.collect()
    return df_temp_sum

In [267]:
def apply_calibration(df, calibration_model, divide_by_score, divide_by_pd, divide_by_cnt, factor, offset):
    
    try:
        interceptCoef = calibration_model.params.const
    except:
        interceptCoef = 0        
    try:
        rowScoreCoef = calibration_model.params.row_score
    except:
        rowScoreCoef = 0
    
    df['calb_raw_score'] = interceptCoef + rowScoreCoef*df['row_score']
    df['calb_pd'] = 1 / (1 + np.exp(-df['calb_raw_score']))
    df['score'] = offset - (np.log(df["predicted"]/(1-df["predicted"]))*factor)
    df['score_grp'] = pd.cut(df['score'], divide_by_score).astype('category')
    df['calb_score'] = offset - (np.log(df["calb_pd"]/(1-df["calb_pd"]))*factor)
    df['calb_score_grp'] = pd.cut(df['calb_score'], divide_by_score).astype('category')
    df['pd_grp'] = pd.cut(df['predicted'], divide_by_pd).astype('category')
    df['calb_pd_grp'] = pd.cut(df['calb_pd'], divide_by_pd).astype('category')
    df = df.sort_values(by='calb_pd', ascending=False)
    df['row'] = np.arange(1, df.shape[0]+1)
    df['calb_row_grp'] = pd.cut(df['row'], divide_by_cnt).astype('category')
    gc.collect()
    return df

In [268]:
def create_performance_measurement_results(df, title):
    df_sum = df.copy()
    temp_score = df_sum.groupby("calb_score_grp", observed=True )["actual", "predicted", "calb_pd", "count","calb_score_grp"].agg("sum").reset_index() 
    temp_score.index = np.arange(1, len(temp_score) + 1)
    
    temp_row = df_sum.groupby("calb_row_grp", observed=True )["actual", "predicted", "calb_pd", "count","calb_score_grp"].agg("sum").reset_index() 
    temp_row.index = np.arange(1, len(temp_row) + 1)
    
    temp_score['act_avg_pd'] = temp_score["actual"]/temp_score["count"]
    temp_score['pred_avg_pd'] = temp_score["predicted"]/temp_score["count"]
    temp_score['calb_avg_pd'] = temp_score["calb_pd"]/temp_score["count"]
    temp_score['dist'] = temp_score["count"]/df_sum.shape[0]
    
    temp_row['act_avg_pd'] = temp_row["actual"]/temp_row["count"]
    temp_row['pred_avg_pd'] = temp_row["predicted"]/temp_row["count"]
    temp_row['calb_avg_pd'] = temp_row["calb_pd"]/temp_row["count"]
    temp_row['dist'] = temp_row["count"]/df_sum.shape[0]
    
    fig_score = plt.figure()
    plt.plot(temp_score["act_avg_pd"], '-r')  # red
    plt.plot(temp_score["pred_avg_pd"], '--b')  # blue
    plt.plot(temp_score["calb_avg_pd"], ':k')  # black
    plt.legend(['Actual', 'Predicted', 'Calibrated'], loc='upper right', shadow=True)
    plt.title(title+" - Average PD by Score")
    plt.xlabel("Score bins")
    plt.ylabel("pd")
    plt.close()
    
    fig_row = plt.figure()
    plt.plot(temp_row["act_avg_pd"], '-r')  # red
    plt.plot(temp_row["pred_avg_pd"], '--b')  # blue
    plt.plot(temp_row["calb_avg_pd"], ':k')  # black
    plt.legend(['Actual', 'Predicted', 'Calibrated'], loc='upper right', shadow=True)
    plt.title(title+" - Average PD by Population Bins")
    plt.xlabel("Population bins")
    plt.ylabel("pd")
    plt.close()
    del df_sum
    gc.collect()
    return temp_score, temp_row, fig_score, fig_row

In [269]:
def set_params(seed):
    params = {
    # Parameters that we are going to tune.
    'eta' : 0.1,
    'max_depth': 6,
    'min_child_weight': 5,
    'subsample': 1,
    'colsample_bytree': 1,
    'gamma' : 10,
    'reg_lambda' :1,
    # Other parameters
    'objective': "binary:logistic",
    'eval_metric' : "auc",
    'booster' : "gbtree",
    'seed' : seed
    }
    return params


In [270]:
def execute_grid_search(df, key_var, grid_params, splitting_perc, min_date, seed, random_search, diff_threshold, grid_size):
    start_time = datetime.now()
    
    train_index, test_index, oot_index = set_split_index(df, splitting_perc, seed, min_date)
    dtrain, dtest, doot, y_train, y_test, y_oot, train_keys, test_keys, oot_keys = split_df_by_index(df, key_var, train_index, test_index, oot_index)
    
    evallist = [(dtrain, 'train'),(dtest, 'eval')]
    
    grid_params = expand_grid(grid_params)
    
    if random_search:
        grid_params = grid_params.sample(n=grid_size, replace = False, random_state=seed)
        
    xgb_gs = pd.DataFrame() #create xgboost grid search dataframe
    
    iter_count = 0
    for i in range(grid_params.shape[0]):
        params = {
            'eta' : grid_params['eta'].iloc[i],
            'max_depth': grid_params['max_depth'].iloc[i],
            'min_child_weight': grid_params['min_child_weight'].iloc[i],
            'subsample': grid_params['subsample'].iloc[i],
            'colsample_bytree': grid_params['colsample_bytree'].iloc[i],
            'gamma' : grid_params['gamma'].iloc[i],  # [10, 20, 30, 40]
            'reg_lambda' : grid_params['reg_lambda'].iloc[i],
            # Other parameters
            'objective': grid_params['objective'].iloc[i],
            'eval_metric' : grid_params['eval_metric'].iloc[i],
            'booster' : grid_params['booster'].iloc[i],
            'seed' : seed           
            }
        iter_count = iter_count+1
        print('\niteration:', iter_count)
        bst = xgb.train(params=params, dtrain=dtrain, num_boost_round=400, evals=evallist, early_stopping_rounds=10, verbose_eval=100 )
        
        pred_train = bst.predict(dtrain, ntree_limit=bst.best_ntree_limit)
        pred_test = bst.predict(dtest, ntree_limit=bst.best_ntree_limit) 
        pred_oot = bst.predict(doot, ntree_limit=bst.best_ntree_limit)
        
        train_df = pd.DataFrame({'sample': 'train', 
                                  'actual': y_train, 
                                  'predicted' : pred_train
                                #  'FM_CUSTOMER_ID':, 
                                #  'REPORT_DATE': , 
                                  })
        test_df = pd.DataFrame({'sample': 'test', 
                                  'actual': y_test, 
                                  'predicted' : pred_test
                                #  'FM_CUSTOMER_ID':, 
                                #  'REPORT_DATE': , 
                                  })
        oot_df = pd.DataFrame({'sample': 'oot', 
                                  'actual': y_oot, 
                                  'predicted' : pred_oot
                                #  'FM_CUSTOMER_ID':, 
                                #  'REPORT_DATE': , 
                                  })
        predictions = train_df.append([test_df, oot_df])
        ##predictions[['actual', 'predicted']] = predictions[['actual', 'predicted']].apply(lambda x: round(100*x, 2))

        auc_train, gini_train = cal_auc_and_gini(predictions, sample='train',pred_name='predicted')
        auc_test, gini_test = cal_auc_and_gini(predictions, sample='test',pred_name='predicted')
        auc_oot, gini_oot = cal_auc_and_gini(predictions, sample='oot', pred_name='predicted')
        
        perf_sum = predictions.groupby("sample").agg("mean").reset_index()
        perf_sum[['actual', 'predicted']] = perf_sum[['actual', 'predicted']].apply(lambda x: round(100*x, 2))
        
        perf_sum['auc'] = 0
        perf_sum['gini'] = 0
        perf_sum.loc[perf_sum['sample'] == 'train', 'auc'] = auc_train
        perf_sum.loc[perf_sum['sample'] == 'test', 'auc'] = auc_test
        perf_sum.loc[perf_sum['sample'] == 'oot', 'auc'] = auc_oot
        perf_sum.loc[perf_sum['sample'] == 'train', 'gini'] = gini_train
        perf_sum.loc[perf_sum['sample'] == 'test', 'gini'] = gini_test
        perf_sum.loc[perf_sum['sample'] == 'oot', 'gini'] = gini_oot
        
        perf_sum['eta'] = grid_params['eta'].iloc[i]
        perf_sum['max_depth'] = grid_params['max_depth'].iloc[i]
        perf_sum['min_child_weight'] = grid_params['min_child_weight'].iloc[i]
        perf_sum['subsample'] = grid_params['subsample'].iloc[i]
        perf_sum['colsample_bytree'] = grid_params['colsample_bytree'].iloc[i]
        perf_sum['gamma'] = grid_params['gamma'].iloc[i]
        perf_sum['reg_lambda'] = grid_params['reg_lambda'].iloc[i]
        perf_sum['objective'] = grid_params['objective'].iloc[i]
        perf_sum['eval_metric'] = grid_params['eval_metric'].iloc[i]
        perf_sum['booster'] = grid_params['booster'].iloc[i]
        perf_sum['grid_iter'] = iter_count
        
        xgb_gs = xgb_gs.append(perf_sum, ignore_index=True)
        
        del auc_train, auc_test, auc_oot, gini_train, gini_test, gini_oot, bst, predictions, train_df, test_df, oot_df, pred_train, pred_test, pred_oot
        gc.collect()
        
    print("\n----selecting best grid iteration----")
    #select best grid results
    xgb_gs_train = xgb_gs.loc[xgb_gs['sample'] == "train"]
    xgb_gs_train = xgb_gs_train.sort_values('gini', ascending=False)
    
    best_gs_iter = np.nan
    xgb_gs_train['trainGini'] = np.nan
    xgb_gs_train['testGini'] = np.nan
    xgb_gs_train['ootGini'] = np.nan
    xgb_gs_train['testDiff'] = np.nan
    xgb_gs_train['ootDiff'] = np.nan
    xgb_gs_train['avgDiff'] = np.nan
    
    for i in xgb_gs_train.index:
        iter_ = xgb_gs_train.loc[i, 'grid_iter']
            
        train_gini = xgb_gs.loc[(xgb_gs['grid_iter'] == iter_) & (xgb_gs['sample']=='train'), 'gini'].iloc[0]
        test_gini = xgb_gs.loc[(xgb_gs['grid_iter'] == iter_) & (xgb_gs['sample']=='test'), 'gini'].iloc[0]
        oot_gini = xgb_gs.loc[(xgb_gs['grid_iter'] == iter_) & (xgb_gs['sample']=='oot'), 'gini'].iloc[0]    
        
        xgb_gs_train.loc[i, 'trainGini'] = train_gini
        xgb_gs_train.loc[i, 'testGini'] = test_gini
        xgb_gs_train.loc[i, 'ootGini'] = oot_gini
        xgb_gs_train.loc[i, 'testDiff'] = abs(train_gini - test_gini)
        xgb_gs_train.loc[i, 'ootDiff'] = abs(train_gini - oot_gini)
        xgb_gs_train.loc[i, 'avgDiff'] = (abs(train_gini - test_gini) + abs(train_gini - oot_gini))/2

    del i, iter_, train_gini, test_gini, oot_gini
    gc.collect()

    xgb_gs_train_base = xgb_gs_train.copy()
    xgb_gs_train = xgb_gs_train.loc[xgb_gs_train['gini'] > 0].copy()
    minAvgDiff = xgb_gs_train['avgDiff'].min()

    if minAvgDiff <= diff_threshold:
        xgb_gs_train_f = xgb_gs_train.loc[xgb_gs_train['avgDiff'] <= diff_threshold].copy()
        max_train_gini = xgb_gs_train_f['gini'].max()
        
        xgb_gs_train_f['StdDev'] = np.nan
        xgb_gs_train_f['trainGiniPrevious'] = max_train_gini
        xgb_gs_train_f['trainGiniPreviousDiff'] = np.nan
        xgb_gs_train_f['finalEvaluationScore'] = np.nan
        
        for i in xgb_gs_train_f.index: 
            train_gini = xgb_gs_train_f.loc[i, 'trainGini']
            test_gini = xgb_gs_train_f.loc[i, 'testGini']
            oot_gini = xgb_gs_train_f.loc[i, 'ootGini']
            xgb_gs_train_f.loc[i, 'StdDev'] = stdev([train_gini,test_gini,oot_gini])
            xgb_gs_train_f.loc[i, 'trainGiniPreviousDiff'] = max_train_gini - train_gini
            xgb_gs_train_f.loc[i, 'finalEvaluationScore'] = ((0.50 * xgb_gs_train_f.loc[i, 'trainGiniPreviousDiff']) + (0.25 * xgb_gs_train_f.loc[i, 'avgDiff']) + (0.25 * xgb_gs_train_f.loc[i, 'StdDev']))
            
            
        xgb_gs_train = pd.merge(xgb_gs_train,xgb_gs_train_f[['grid_iter','StdDev', 'trainGiniPrevious', 'trainGiniPreviousDiff','finalEvaluationScore']],on='grid_iter', how='left')
        min_final_eva_score = copy.copy(xgb_gs_train_f['finalEvaluationScore'].min())
        best_gs_iter = xgb_gs_train_f.loc[xgb_gs_train_f['finalEvaluationScore'] <= min_final_eva_score, 'grid_iter'].iloc[0]
        xgb_gs_train = pd.merge(xgb_gs_train_base,xgb_gs_train[['grid_iter','StdDev', 'trainGiniPrevious', 'trainGiniPreviousDiff','finalEvaluationScore']],on='grid_iter', how='left')
        del i, train_gini, test_gini, oot_gini
        gc.collect()
    else:
        min_diff = xgb_gs_train['avgDiff'].min()
        min_diff_gini = xgb_gs_train.loc[xgb_gs_train['avgDiff'] <= minAvgDiff, 'gini'].iloc[0]
        for i in xgb_gs_train.index:
            #iter_ = j['grid_iter']
            diff_increase = xgb_gs_train.loc[i, 'avgDiff'] - min_diff
            gini_decrease = xgb_gs_train.loc[i, 'gini'] - min_diff_gini
            xgb_gs_train.loc[i, 'finalEvaluationScore'] = 5*diff_increase - gini_decrease
        
        min_final_eva_score = xgb_gs_train['finalEvaluationScore'].min()
        best_gs_iter = xgb_gs_train.loc[xgb_gs_train['finalEvaluationScore'] <= min_final_eva_score, 'grid_iter'].iloc[0]
        xgb_gs_train = xgb_gs_train.reset_index(drop=True)
        xgb_gs_train = pd.merge(xgb_gs_train_base,xgb_gs_train[['grid_iter','finalEvaluationScore']],on='grid_iter', how='left')
        del i, diff_increase, gini_decrease, min_diff, min_diff_gini
        gc.collect()
    print("best grid iteration:", best_gs_iter)
    
    params = {
    # Parameters that we are going to tune.
    'eta' : xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'eta'].iloc[0],
    'max_depth': xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'max_depth'].iloc[0],
    'min_child_weight': xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'min_child_weight'].iloc[0],
    'subsample': xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'subsample'].iloc[0],
    'colsample_bytree': xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'colsample_bytree'].iloc[0],
    'gamma' : xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'gamma'].iloc[0],
    'reg_lambda' : xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'reg_lambda'].iloc[0],
    # Other parameters
    'objective': xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'objective'].iloc[0],
    'eval_metric' : xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'eval_metric'].iloc[0],
    'booster' : xgb_gs_train.loc[xgb_gs_train['grid_iter'] == best_gs_iter,'booster'].iloc[0],
    'seed' : seed
    }
    xgb_gs_train[['actual', 'predicted']] = xgb_gs_train[['actual', 'predicted']].apply(lambda x: round(x, 2))
    end_time = datetime.now()
    print('Grid Search duration: {}'.format(end_time - start_time))
    return xgb_gs_train, best_gs_iter, params

In [271]:
def execute_final_model(df, key_var, params, splitting_perc, seed, num_boost_round, early_stopping_rounds, verbose_eval):
    print("\n----final model----")
    train_index, test_index, oot_index = set_split_index(df, splitting_perc, seed, min_date)
    dtrain, dtest, doot, y_train, y_test, y_oot, train_keys, test_keys, oot_keys = split_df_by_index(df, key_var, train_index, test_index, oot_index)
    print("\n")
    
    evallist = [(dtrain, 'train'),(dtest, 'eval')]
    
    bst = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round, evals=evallist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose_eval )
    
    pred_train = bst.predict(dtrain, ntree_limit=bst.best_ntree_limit)
    pred_test = bst.predict(dtest, ntree_limit=bst.best_ntree_limit) 
    pred_oot = bst.predict(doot, ntree_limit=bst.best_ntree_limit)
    
    train_df = pd.DataFrame({'sample': 'train', 
                              'actual': y_train, 
                              'predicted' : pred_train,
                              'APP_ID': train_keys["APP_ID"], 
                              'APPLICATION_DATE': train_keys["APPLICATION_DATE"] 
                              })
    
    test_df = pd.DataFrame({'sample': 'test', 
                             'actual': y_test, 
                             'predicted' : pred_test,
                             'APP_ID': test_keys["APP_ID"], 
                             'APPLICATION_DATE': test_keys["APPLICATION_DATE"] 
                              })
    oot_df = pd.DataFrame({'sample': 'oot', 
                            'actual': y_oot, 
                            'predicted' : pred_oot,
                            'APP_ID': oot_keys["APP_ID"], 
                             'APPLICATION_DATE': oot_keys["APPLICATION_DATE"]
                              })
    predictions = train_df.append([test_df, oot_df])
    del train_df, test_df, oot_df, pred_train, pred_test, pred_oot
    gc.collect()
    perf_sum = predictions.groupby("sample").agg("mean").reset_index().drop(columns='APP_ID')
    perf_sum[['actual', 'predicted']] = perf_sum[['actual', 'predicted']].apply(lambda x: round(100*x, 2))
    
    ### Calibration ###
    
    divide_by_score = np.arange(0, 1001, 10)
    divide_by_pd = 10
    divide_by_cnt = 10
    factor = 20/np.log(2)
    offset = 500 - factor*np.log(50)
    
    train_sum = create_calibration_df(predictions, sample='train')
    test_sum = create_calibration_df(predictions, sample='test')
    oot_sum = create_calibration_df(predictions, sample='oot')
    
    calibration_model = sm.GLM(train_sum.actual, sm.add_constant(train_sum.row_score), family=sm.families.Binomial()).fit()
    
    train_sum = apply_calibration(train_sum, calibration_model, divide_by_score, divide_by_pd, divide_by_cnt, factor, offset)
    test_sum = apply_calibration(test_sum, calibration_model, divide_by_score, divide_by_pd, divide_by_cnt, factor, offset)
    oot_sum = apply_calibration(oot_sum, calibration_model, divide_by_score, divide_by_pd, divide_by_cnt, factor, offset)
    
    predictions_calb = train_sum.append([test_sum, oot_sum])
    
    auc_train, gini_train = cal_auc_and_gini(train_sum, sample='train', pred_name='predicted')
    auc_test, gini_test = cal_auc_and_gini(test_sum, sample='test', pred_name='predicted')
    auc_oot, gini_oot = cal_auc_and_gini(oot_sum, sample='oot', pred_name='predicted')
    auc_train_calb, gini_train_calb = cal_auc_and_gini(train_sum, sample='train',pred_name='calb_pd')
    auc_test_calb, gini_test_calb = cal_auc_and_gini(test_sum, sample='test',pred_name='calb_pd')
    auc_oot_calb, gini_oot_calb = cal_auc_and_gini(oot_sum, sample='oot',pred_name='calb_pd')
    
    
    ### Performance Summary ###
    perf_sum["calibrated"] = np.nan
    # case_when
    conditions = [(perf_sum['sample'].eq("train")), 
                  (perf_sum['sample'].eq("test")),
                  (perf_sum['sample'].eq("oot"))]
    choices = [np.mean(train_sum['calb_pd']), np.mean(test_sum['calb_pd']), np.mean(oot_sum['calb_pd'])]
    perf_sum["calibrated"] = np.select(conditions, choices)
    
    perf_sum["calibrated"] = perf_sum["calibrated"].apply(lambda x: round(100*x, 2))
    
    perf_sum['auc'] = 0
    perf_sum['auc_calb'] = 0
    perf_sum.loc[perf_sum['sample'] == 'train', 'auc'] = auc_train
    perf_sum.loc[perf_sum['sample'] == 'test', 'auc'] = auc_test
    perf_sum.loc[perf_sum['sample'] == 'oot', 'auc'] = auc_oot
    perf_sum.loc[perf_sum['sample'] == 'train', 'gini'] = gini_train
    perf_sum.loc[perf_sum['sample'] == 'test', 'gini'] = gini_test
    perf_sum.loc[perf_sum['sample'] == 'oot', 'gini'] = gini_oot
    
    perf_sum.loc[perf_sum['sample'] == 'train', 'auc_calb'] = auc_train_calb
    perf_sum.loc[perf_sum['sample'] == 'test', 'auc_calb'] = auc_test_calb
    perf_sum.loc[perf_sum['sample'] == 'oot', 'auc_calb'] = auc_oot_calb
    perf_sum.loc[perf_sum['sample'] == 'train', 'gini_calb'] = gini_train_calb
    perf_sum.loc[perf_sum['sample'] == 'test', 'gini_calb'] = gini_test_calb
    perf_sum.loc[perf_sum['sample'] == 'oot', 'gini_calb'] = gini_oot_calb
    
    save_params = params.copy()
    save_params["num_boost_round"] = num_boost_round
    save_params["early_stopping_rounds"] = early_stopping_rounds
    save_params["verbose_eval"] = verbose_eval
    perf_sum["parameters"] = str(save_params)
    
    ## feature importance 
    
    # ‘weight’: the number of times a feature is used to split the data across all trees.
    # ‘gain’: the average gain across all splits the feature is used in.
    # ‘cover’: the average coverage across all splits the feature is used in.
    # ‘total_gain’: the total gain across all splits the feature is used in.
    # ‘total_cover’: the total coverage across all splits the feature is used in.
    
    importance_xgb = pd.DataFrame.from_dict(data=bst.get_score(importance_type='weight'), orient='index')
    importance_xgb = importance_xgb.reset_index()
    importance_xgb.columns = ["features","weight"]
    
    for i in ['gain', 'cover', 'total_gain', 'total_cover']:
        temp_imp = pd.DataFrame.from_dict(data=bst.get_score(importance_type=i), orient='index')
        temp_imp.columns = [i]
        importance_xgb = importance_xgb.merge(temp_imp, left_on="features", right_index=True)
        del temp_imp
        gc.collect()
        
    importance_xgb["total_gain_score"] =  importance_xgb["total_gain"] / importance_xgb["total_gain"].sum()
    importance_xgb = importance_xgb.sort_values('total_gain_score', ascending=False)
    del auc_train, auc_test, auc_oot, gini_train, gini_test, gini_oot, save_params
    del auc_train_calb, auc_test_calb, auc_oot_calb, gini_train_calb, gini_test_calb, gini_oot_calb
    gc.collect()
    return bst, predictions, predictions_calb, train_sum, test_sum, oot_sum, perf_sum, importance_xgb,calibration_model


In [272]:
def performance_measurement():
    # performance measurement
    train_sum_by_calb_score, train_sum_by_calb_row, fig_train_score, fig_train_row = create_performance_measurement_results(train_sum, title="Train")
    test_sum_by_calb_score, test_sum_by_calb_row, fig_test_score, fig_test_row = create_performance_measurement_results(test_sum, title="Test")
    oot_sum_by_calb_score, oot_sum_by_calb_row, fig_oot_score, fig_oot_row = create_performance_measurement_results(oot_sum, title="Oot")
    return train_sum_by_calb_score, train_sum_by_calb_row, fig_train_score, fig_train_row, test_sum_by_calb_score, test_sum_by_calb_row, fig_test_score, fig_test_row, oot_sum_by_calb_score, oot_sum_by_calb_row, fig_oot_score, fig_oot_row



In [273]:
def calculate_auc_gini_for_segment(dfPredictions, segmentType, segmentTypeGroup, pred_name):
  
    try:
        temp = dfPredictions.copy()
        
        if segmentType == "all":
           auc_ = roc_auc_score(temp['actual'], temp[pred_name])
        else:
           auc_ = roc_auc_score(temp.loc[temp[segmentType] == segmentTypeGroup]['actual'], temp.loc[temp[segmentType] == segmentTypeGroup][pred_name])
           
        gini_ = round((2*auc_ - 1)*100, 2)
        auc_ = round(auc_*100, 2)
    except:    
        print('Gini is not calculated for:' , segmentType, segmentTypeGroup)
        gini_ = "not calculated"
        auc_ = "not calculated"

    del temp
    gc.collect()
    
    return auc_,gini_


In [274]:
def evaluate_performance_by_segment():
    # performance measurement by different segmentation scenarios    
    
    dfSegmentEvaluation = predictions_calb.merge(dfSegmentation, on=['APP_ID','APPLICATION_DATE'], how='left')
    
    segment_summary_all_sample = dfSegmentEvaluation.groupby("count", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum")
    segment_summary_all_sample.insert(loc = 0, column = 'segmentType', value = "all")
    segment_summary_all_sample.insert(loc = 0, column = 'segmentTypeGroup', value = "all")
    
    segment_summary_customer_type = dfSegmentEvaluation.groupby("segmentCustomerType", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_customer_type.insert(loc = 0, column = 'segmentType', value = "segmentCustomerType")
    segment_summary_customer_type.rename(columns={'segmentCustomerType':'segmentTypeGroup'}, inplace = True) 

    segment_summary_financial_type = dfSegmentEvaluation.groupby("segmentFinancialAvailable", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_financial_type.insert(loc = 0, column = 'segmentType', value = "segmentFinancialAvailable")
    segment_summary_financial_type.rename(columns={'segmentFinancialAvailable':'segmentTypeGroup'}, inplace = True)

    segment_summary_customer_tenure = dfSegmentEvaluation.groupby("segmentCustomerTenure", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_customer_tenure.insert(loc = 0, column = 'segmentType', value = "segmentCustomerTenure")
    segment_summary_customer_tenure.rename(columns={'segmentCustomerTenure':'segmentTypeGroup'}, inplace = True)

    segment_summary_credit_tenure = dfSegmentEvaluation.groupby("segmentCreditTenure", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_credit_tenure.insert(loc = 0, column = 'segmentType', value = "segmentCreditTenure")
    segment_summary_credit_tenure.rename(columns={'segmentCreditTenure':'segmentTypeGroup'}, inplace = True)

    segment_summary_sector_type = dfSegmentEvaluation.groupby("segmentSectorType", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_sector_type.insert(loc = 0, column = 'segmentType', value = "segmentSectorType")
    segment_summary_sector_type.rename(columns={'segmentSectorType':'segmentTypeGroup'}, inplace = True)

    segment_summary_limit_level = dfSegmentEvaluation.groupby("segmentLimitLevel", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_limit_level.insert(loc = 0, column = 'segmentType', value = "segmentLimitLevel")
    segment_summary_limit_level.rename(columns={'segmentLimitLevel':'segmentTypeGroup'}, inplace = True)

    segment_summary_netsales_level = dfSegmentEvaluation.groupby("segmentNetSalesLevel", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_netsales_level.insert(loc = 0, column = 'segmentType', value = "segmentNetSalesLevel")
    segment_summary_netsales_level.rename(columns={'segmentNetSalesLevel':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_mrt = dfSegmentEvaluation.groupby("segmentBireyselMrt", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_mrt.insert(loc = 0, column = 'segmentType', value = "segmentBireyselMrt")
    segment_summary_bireysel_mrt.rename(columns={'segmentBireyselMrt':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_veh = dfSegmentEvaluation.groupby("segmentBireyselVeh", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_veh.insert(loc = 0, column = 'segmentType', value = "segmentBireyselVeh")
    segment_summary_bireysel_veh.rename(columns={'segmentBireyselVeh':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_gpl = dfSegmentEvaluation.groupby("segmentBireyselGpl", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_gpl.insert(loc = 0, column = 'segmentType', value = "segmentBireyselGpl")
    segment_summary_bireysel_gpl.rename(columns={'segmentBireyselGpl':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_cc = dfSegmentEvaluation.groupby("segmentBireyselCC", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_cc.insert(loc = 0, column = 'segmentType', value = "segmentBireyselCC")
    segment_summary_bireysel_cc.rename(columns={'segmentBireyselCC':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_od = dfSegmentEvaluation.groupby("segmentBireyselOD", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_od.insert(loc = 0, column = 'segmentType', value = "segmentBireyselOD")
    segment_summary_bireysel_od.rename(columns={'segmentBireyselOD':'segmentTypeGroup'}, inplace = True)

    segment_summary_bireysel_kkb = dfSegmentEvaluation.groupby("segmentFMBireyselKKB", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_bireysel_kkb.insert(loc = 0, column = 'segmentType', value = "segmentFMBireyselKKB")
    segment_summary_bireysel_kkb.rename(columns={'segmentFMBireyselKKB':'segmentTypeGroup'}, inplace = True)

    segment_summary_kurumsal_kkb = dfSegmentEvaluation.groupby("segmentFMKurumsalKKB", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_kurumsal_kkb.insert(loc = 0, column = 'segmentType', value = "segmentFMKurumsalKKB")
    segment_summary_kurumsal_kkb.rename(columns={'segmentFMKurumsalKKB':'segmentTypeGroup'}, inplace = True)

    dfSegmentEvaluation['segmentCleanDirty'] = np.where(dfSegmentEvaluation['segmentPaymHistoryL12M']==0, 'clean', 'dirty')
    segment_summary_clean_dirty = dfSegmentEvaluation.groupby("segmentCleanDirty", observed=True )["actual", "predicted", "calb_pd", "count"].agg("sum").reset_index() 
    segment_summary_clean_dirty.insert(loc = 0, column = 'segmentType', value = "segmentCleanDirty")
    segment_summary_clean_dirty.rename(columns={'segmentCleanDirty':'segmentTypeGroup'}, inplace = True)

    segment_summary_all = segment_summary_all_sample.append([segment_summary_customer_type, segment_summary_financial_type,segment_summary_customer_tenure,segment_summary_credit_tenure,segment_summary_sector_type, segment_summary_limit_level,segment_summary_netsales_level,segment_summary_bireysel_mrt,segment_summary_bireysel_veh,segment_summary_bireysel_gpl, segment_summary_bireysel_cc, segment_summary_bireysel_od, segment_summary_bireysel_kkb, segment_summary_kurumsal_kkb, segment_summary_clean_dirty])
    del segment_summary_all_sample, segment_summary_customer_type, segment_summary_financial_type,segment_summary_customer_tenure,segment_summary_credit_tenure,segment_summary_sector_type, segment_summary_limit_level,segment_summary_netsales_level,segment_summary_bireysel_mrt,segment_summary_bireysel_veh,segment_summary_bireysel_gpl, segment_summary_bireysel_cc, segment_summary_bireysel_od, segment_summary_bireysel_kkb, segment_summary_kurumsal_kkb,segment_summary_clean_dirty
    
    segment_summary_all["avg_pd"] = segment_summary_all["predicted"]/segment_summary_all["count"]
    segment_summary_all["avg_calb_pd"] = segment_summary_all["calb_pd"]/segment_summary_all["count"]
    
    segment_summary_all["auc"] = np.nan
    segment_summary_all["auc_calb"] = np.nan
    segment_summary_all["gini"] = np.nan
    segment_summary_all["gini_calb"] = np.nan
    
    for segmentType in pd.unique(segment_summary_all['segmentType']):
        for segmentTypeGroup in pd.unique(segment_summary_all.loc[segment_summary_all['segmentType']== segmentType]['segmentTypeGroup']):
            aucSegment,giniSegment = calculate_auc_gini_for_segment(dfSegmentEvaluation, segmentType, segmentTypeGroup, "predicted")
            aucSegment_calb, giniSegment_calb = calculate_auc_gini_for_segment(dfSegmentEvaluation, segmentType, segmentTypeGroup, "calb_pd")
            segment_summary_all.loc[(segment_summary_all['segmentType'] == segmentType) & (segment_summary_all['segmentTypeGroup'] == segmentTypeGroup), 'auc'] = aucSegment
            segment_summary_all.loc[(segment_summary_all['segmentType'] == segmentType) & (segment_summary_all['segmentTypeGroup'] == segmentTypeGroup), 'auc_calb'] = aucSegment_calb
            segment_summary_all.loc[(segment_summary_all['segmentType'] == segmentType) & (segment_summary_all['segmentTypeGroup'] == segmentTypeGroup), 'gini'] = giniSegment
            segment_summary_all.loc[(segment_summary_all['segmentType'] == segmentType) & (segment_summary_all['segmentTypeGroup'] == segmentTypeGroup), 'gini_calb'] = giniSegment_calb
    
    return segment_summary_all



In [275]:
####################
### Save Results ###
####################

def save_results(path):
    print("\nSaving Results...")
    # create folder
    if not os.path.exists(path):
        os.mkdir(path)
    
    # save grid search results
    xgb_gs_train.to_excel(path + "fullgrid.xlsx" )
    
    # save predictions
    predictions_calb.to_csv(path + "predictions_calb.csv" )
    
    # save performance results
    perf_sum.to_excel(path + "performance_summary.xlsx" )
    
    # save comparison plot - actual vs predicted
    with matplotlib.backends.backend_pdf.PdfPages(path +'Performance_Details.pdf') as pdf:
        for i in [fig_train_score, fig_train_row, fig_test_score, fig_test_row, fig_oot_score, fig_oot_row]:
            pdf.savefig(i)
        
    # save importance features' information
    importance_xgb.to_excel(path +"importance_xgb.xlsx")   
    
    
    # save model performance summary data
    perf_dict = {'train_score': train_sum_by_calb_score, 'train_count': train_sum_by_calb_row,
                 'test_score': test_sum_by_calb_score, 'test_count': test_sum_by_calb_row,
                 'oot_score': oot_sum_by_calb_score, 'oot_count': oot_sum_by_calb_row}
    
    with pd.ExcelWriter(path +'Performance_Details_data.xlsx') as writer:
        for name_ in perf_dict:
            perf_dict[name_].to_excel(writer, sheet_name=name_)
    
    del perf_dict
    gc.collect()
    
    # save gini performance by various segment breakdown
    #segment_summary_all.to_excel(path +"segment_breakdown.xlsx")

            
    # save xgb model object
    pickle.dump(bst, open(path +"model_xgb.pkl", 'wb'))
    # load the xgb model from disk
    #loaded_model = pickle.load(open(path+filename, 'rb'))
    
    # save initial calibration object
    #calibration_model.save(path /"calibration_model.pkl")
    pickle.dump(calibration_model, open(path +"calibration_model.pkl", 'wb'))
    
    #sklearn2pmml(pipeline, path + str("xgboost_model.pmml"), with_repr = True)
    # load the calibration model from disk
    #loaded_calb = sm.load(path+"calibration_model.pkl")

    # save character encoding matrix
    #encoding_matrix.to_excel(path +"encoding_matrix.xlsx")
    #encoding_matrix.to_pickle(path +"encoding_matrix.pkl")


In [276]:
def execute_preprocess(df, max_threshold):
    '''
    execute all functions defined for preprocessing
    '''
    na_report = create_na_report(df)
    num_cols, char_cols = create_dtype_report(df)
    new_df = eliminate_missing(df, na_report, max_threshold)
    new_df = eliminate_single_unique(new_df)
    #enc_df = create_encoding_matrix(new_df)
    #new_df = encode_from_matrix(new_df, enc_df)
    
    #save enc_df for later use in prediction code
    gc.collect()
    print("\nPreprocess is done...")
    return new_df#, enc_df

In [277]:
key_var = ["APP_ID", "APPLICATION_DATE", "TARGET"]

In [278]:
data

,APPLICATION_DATE,Unnamed: 0,APP_ID,age,bp,sg,al,su,rbc,pc,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,TARGET
0,2021-06-01,0,0,48.0,80.0,1.020,1.0,0.0,2,1,...,32,72,34,1,4,1,0,0,0,0
1,2021-06-02,1,1,7.0,50.0,1.020,4.0,0.0,2,1,...,26,56,49,0,3,1,0,0,0,1
2,2021-06-03,2,2,62.0,80.0,1.010,2.0,3.0,1,1,...,19,70,49,0,4,1,1,0,1,1
3,2021-06-04,3,3,48.0,70.0,1.005,4.0,0.0,1,0,...,20,62,19,1,3,1,1,1,1,1
4,2021-06-05,4,4,51.0,80.0,1.010,2.0,0.0,1,1,...,23,68,27,0,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2022-06-29,395,395,55.0,80.0,1.020,0.0,0.0,1,1,...,35,62,30,0,3,1,0,0,0,1
394,2022-06-30,396,396,42.0,70.0,1.025,0.0,0.0,1,1,...,42,72,44,0,3,1,0,0,0,1
395,2022-07-01,397,397,12.0,80.0,1.020,0.0,0.0,1,1,...,37,61,36,0,3,1,0,0,0,0
396,2022-07-02,398,398,17.0,60.0,1.025,0.0,0.0,1,1,...,39,67,41,0,3,1,0,0,0,1


In [286]:
if __name__ == "__main__":
    """main parameters should be set before each run"""
    grid_search_= True
    save_results_= True
    random_search = False
        
    # define segment    
    #if segment in ('BIREYSEL','ORTAK'):
       #random_search = True
    
    # key variables
    key_var = ["APP_ID", "APPLICATION_DATE", "TARGET"] # for management of data issues
    key_vars = ["APP_ID", "APPLICATION_DATE"] # for preparing data.table() for model estimation
    
    # define maximum acceptable percentage of missing information
    max_threshold = 1
    
    # splitting percentage of development data into train nand test
    splitting_perc = 0.2
    min_date = '2022-05-31'
    
     # specify seed for random selection - to keep track of the chosen seed, for results' reproduction
    seed = 42
    
    ## diff_threshold for grid search
    diff_threshold = 5
    segment = 'Deneme'
    trial = '1'
    t_string = str(trial)
    today = date.today()    
    path = 'path'
    
    path = path.format(segment, t_string, today)
    #data = data.drop(columns = 'Unnamed: 0')
    df = data.copy()
    #code execution:
    df = execute_preprocess(df, max_threshold)
    #df = pd.DataFrame(df)
    #df = df.reset_index(drop = True)
    train_index, test_index, oot_index=set_split_index(df, splitting_perc, seed, min_date)
    
    if grid_search_:
        grid_params = {
        # Parameters that we are going to tune.
        'eta' : [0.3, 0.1, 0.05],
        'max_depth': np.arange(3, 11, 2).tolist(),
        'min_child_weight': [round(train_index.shape[0]*0.001),round(train_index.shape[0]*0.003),round(train_index.shape[0]*0.006),round(train_index.shape[0]*0.008),round(train_index.shape[0]*0.01),round(train_index.shape[0]*0.02)],
        'subsample': [1],
        'colsample_bytree': [1],
        'gamma' : [10, 20, 30, 40],
        'reg_lambda' : np.arange(1, 5, 2).tolist(),
        # Other parameters
        'objective': ['reg:logistic'],
        'eval_metric' : ['auc'],
        'booster' : ['gbtree']
        }
        xgb_gs_train, best_gs_iter, params = execute_grid_search(df, key_var, grid_params, splitting_perc, min_date, seed, random_search, diff_threshold, 200)    
    else:
        params = set_params(seed)

    bst, predictions, predictions_calb, train_sum, test_sum, oot_sum, perf_sum, importance_xgb,calibration_model = execute_final_model(df, key_var, params, splitting_perc, seed, num_boost_round=400, early_stopping_rounds=10, verbose_eval=10)
    train_sum_by_calb_score, train_sum_by_calb_row, fig_train_score, fig_train_row, test_sum_by_calb_score, test_sum_by_calb_row, fig_test_score, fig_test_row, oot_sum_by_calb_score, oot_sum_by_calb_row, fig_oot_score, fig_oot_row = performance_measurement()    
    #segment_summary_all = evaluate_performance_by_segment()  
    
    print(perf_sum[["sample","auc"]])
    if save_results_:
        save_results(path)
    print('Done with model_building.py')




Dataset has 398 entries and 27 features
0 columns eliminated due to missing values
0 columns eliminated due to single unique values, namely: []

Preprocess is done...

Shape of dataframes; train: (291,), test: (73,), oot: (34,)

Percentage of dataframes; train: 73.12%, test: 18.34%, oot: 8.54%

Shape of dataframes; train: (291,), test: (73,), oot: (34,)

Percentage of dataframes; train: 73.12%, test: 18.34%, oot: 8.54%

iteration: 1
[0]	train-auc:0.50000	eval-auc:0.50000
[9]	train-auc:0.50000	eval-auc:0.50000


C:\Users\ENES\anaconda3\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



iteration: 2
[0]	train-auc:0.50000	eval-auc:0.50000
[10]	train-auc:0.50000	eval-auc:0.50000


C:\Users\ENES\anaconda3\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



iteration: 3
[0]	train-auc:0.50000	eval-auc:0.50000
[9]	train-auc:0.50000	eval-auc:0.50000


...


iteration: 576
[0]	train-auc:0.50000	eval-auc:0.50000
[9]	train-auc:0.50000	eval-auc:0.50000


C:\Users\ENES\anaconda3\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



----selecting best grid iteration----
best grid iteration: 289
Grid Search duration: 0:03:33.485049

----final model----

Shape of dataframes; train: (291,), test: (73,), oot: (34,)

Percentage of dataframes; train: 73.12%, test: 18.34%, oot: 8.54%


[0]	train-auc:0.70541	eval-auc:0.49808
[9]	train-auc:0.70541	eval-auc:0.49808


C:\Users\ENES\anaconda3\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


  sample    auc
0    oot  47.06
1   test  49.81
2  train  70.54

Saving Results...
Done with model_building.py
